In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0303 16:40:24.091618 4312933824 deprecation.py:323] From <ipython-input-1-3c4f46c1a93b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0303 16:40:24.092931 4312933824 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0303 16:40:24.093718 4312933824 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz


W0303 16:40:24.329834 4312933824 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
for folder in ['out/', 'model/']:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [4]:
mb_size = 32
Z_dim = 64
X_dim = mnist.train.images.shape[1] # 784
m = 5 # 極大值

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, X_dim]))
D_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def discriminator(X): # AutoEncoder
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    X_recon = tf.matmul(D_h1, D_W2) + D_b2
    mse = tf.reduce_mean(tf.reduce_sum((X - X_recon)**2, 1))
    return mse # 

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = D_real + tf.maximum(0., m - D_fake) # 
G_loss = D_fake

pre_D_solver = tf.train.AdamOptimizer(learning_rate=0.001).minimize(D_real, var_list=theta_D) # 用 X_real 預訓練 Discriminator
D_solver = tf.train.AdamOptimizer(learning_rate=0.001).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=0.001).minimize(G_loss, var_list=theta_G)

W0303 16:40:24.747897 4312933824 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
# saver.restore(sess, "model/save_net.ckpt")

In [6]:
i=0
for it in range(10000):
    X_mb, _ =  mnist.train.next_batch(mb_size)
    _, D_l = sess.run([pre_D_solver, D_real], feed_dict={X:X_mb})
    if it%1000==0:
        print('pretrain iter:{} [D loss:{:.4}]'.format(it, D_l))
for it in range(1000000):   
    X_mb, _ =  mnist.train.next_batch(mb_size)
    _, D_l = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:sample_Z(mb_size, Z_dim)})
        
    _, G_l = sess.run([G_solver, G_loss], feed_dict={Z:sample_Z(mb_size, Z_dim)})
    
    if it%1000==0:
        print('iter:{} [D loss:{:.4}] [G loss:{:.4}]'.format(it, D_l, G_l))
        
    if it%1000==0:
        samples = sess.run(G_sample, feed_dict={Z:sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        i+=1
        
    if it%1000==0:
        save_path = saver.save(sess, "model/save_net.ckpt")

pretrain iter:0 [D loss:253.1]
pretrain iter:1000 [D loss:28.14]
pretrain iter:2000 [D loss:26.86]
pretrain iter:3000 [D loss:23.53]
pretrain iter:4000 [D loss:23.69]
pretrain iter:5000 [D loss:23.47]
pretrain iter:6000 [D loss:22.97]
pretrain iter:7000 [D loss:22.11]
pretrain iter:8000 [D loss:23.46]
pretrain iter:9000 [D loss:20.03]
iter:0 [D loss:19.34] [G loss:145.1]
iter:1000 [D loss:19.99] [G loss:5.225]
iter:2000 [D loss:22.8] [G loss:4.245]
iter:3000 [D loss:22.8] [G loss:4.556]
iter:4000 [D loss:22.37] [G loss:4.502]
iter:5000 [D loss:24.65] [G loss:3.99]
iter:6000 [D loss:23.35] [G loss:4.089]
iter:7000 [D loss:21.22] [G loss:5.501]


E0303 16:41:03.183802 4312933824 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-6f190e32bb3f>", line 9, in <module>
    _, D_l = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:sample_Z(mb_size, Z_dim)})
  File "/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 930, in run
    run_metadata_ptr)
  File "/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1153, in _run
    feed_dict_tensor, options, run_metadata)
  File "/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1329, in _do_run
    run_metadata)
  File "/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1335, in _do_call
    

KeyboardInterrupt: 